In [12]:
path = '/kaggle/input/web-server-access-logs/access.log'

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
pip install drain3

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
from drain3 import TemplateMiner
from drain3.template_miner_config import TemplateMinerConfig
from drain3.file_persistence import FilePersistence
import json

# Regex như bạn cung cấp
log_pattern = re.compile(
    r'(?P<ip>\S+)\s+'
    r'(?P<logname>\S+)\s+'
    r'(?P<user>\S+)\s+'
    r'\[(?P<timestamp>[^\]]+)\]\s+'
    r'"(?:(?P<method>[A-Z]+)\s+(?P<url>[^\s"]*)\s+(?P<protocol>[^"]*)|[^"]*)"\s+'
    r'(?P<status>\d{3}|-)\s+'
    r'(?P<size>\d+|-)\s+'
    r'"(?P<referrer>[^"]*)"\s+'
    r'"(?P<user_agent>[^"]*)"'
)

# Không cần load_default_config nữa
config = TemplateMinerConfig()
config.profiling_enabled = False
config.drain_depth = 4
config.similarity_threshold = 0.4
config.max_children = 100

persistence = FilePersistence("drain3_state.json")
template_miner = TemplateMiner(persistence, config)
# Xử lý từng dòng log
parsed_logs = []

with open(path, "r", encoding="utf-8") as f:  # Đổi tên file log nếu khác
    for line in f:
        line = line.strip()
        # Áp dụng regex để lấy field cụ thể
        match = log_pattern.match(line)
        if not match:
            continue  # skip nếu không match
        log_fields = match.groupdict()

        # Dùng Drain3 để tạo template
        result = template_miner.add_log_message(line)

        log_fields["drain3_template"] = result["template_mined"] if result else None
        log_fields["event_id"] = result["cluster_id"] if result else None
        parsed_logs.append(log_fields)

In [ ]:
# stores output in parsed_log.csv
import pandas as pd
df = pd.DataFrame(parsed_logs)


In [ ]:
df.sample(3)

In [ ]:
# Giả sử df đã được tải và chứa cột 'timestamp'

# Tạo bản sao nông để tránh thay đổi df gốc
df_test = df.copy(deep=False)

# Parse timestamp và lưu vào cột mới
df_test['timestamp_dt_test'] = pd.to_datetime(df_test['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')


In [ ]:
from IPython.display import FileLink

# Save file
df.to_csv("parsed_log.csv", index=False)
FileLink("parsed_log.csv")

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
top_event_ids = df['event_id'].value_counts().head(10)
sns.barplot(x=top_event_ids.values, y=top_event_ids.index)
plt.title("Top 10 Frequent Log Templates (event_id)")

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df[df["event_id"] == 4695])